In [81]:
import numpy as np
import pandas as pd
from apyori import apriori

In [82]:
df = pd.read_csv('fanituotteet.csv', sep =',')

In [83]:
df.shape

(100000, 21)

In [84]:
colnames = df.columns
for colname in colnames:
    print (colname)

ID
MovieBoxes
TShirts
CollegeShirts
Caps
Hats
Mugs
Music
Posters
Books
Scarves
PintsAndWhiskeyGlasses
ScaleModels
CollectorCards
BoardGames
FlagsAndStandards
Stickers
Toys
Sweets
FridgeMagnets
UnderwearAndSocks


In [85]:
transactions = [] # kaikki = 'kuitti'
for i in range(0, df.shape[0]):
    transact = [] # yksittäinen rivi kuitilla
    for j in range(1, df.shape[1]):
        if df.values[i, j] == 1:
            transact.append(colnames[j])
    if len(transact) > 0:
        transactions.append(transact) # ei lisätä tyhiä rivejä
print(transactions[0:10]) # listoja listan sisällä



[['Books', 'FlagsAndStandards', 'UnderwearAndSocks'], ['TShirts', 'Books', 'FlagsAndStandards', 'Stickers', 'Toys', 'Sweets', 'FridgeMagnets', 'UnderwearAndSocks'], ['Music', 'FridgeMagnets', 'UnderwearAndSocks'], ['MovieBoxes', 'Caps', 'PintsAndWhiskeyGlasses', 'FridgeMagnets', 'UnderwearAndSocks'], ['Books', 'FlagsAndStandards', 'FridgeMagnets', 'UnderwearAndSocks'], ['TShirts', 'Music', 'PintsAndWhiskeyGlasses'], ['Sweets', 'FridgeMagnets', 'UnderwearAndSocks'], ['Posters', 'Stickers'], ['ScaleModels', 'FridgeMagnets', 'UnderwearAndSocks'], ['Books', 'CollectorCards']]


In [86]:
results = list(apriori(transactions = transactions,
                       min_support = 0.1,
                       min_confidence = 0.1,
                       min_lift = 1.1,
                       min_length = 1,
                       max_length = 5
                       ))

for result in results:
    print(results)


[RelationRecord(items=frozenset({'FlagsAndStandards', 'Books'}), support=0.13848161033797216, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Books'}), items_add=frozenset({'FlagsAndStandards'}), confidence=0.5613761144411423, lift=3.8029923981669307), OrderedStatistic(items_base=frozenset({'FlagsAndStandards'}), items_add=frozenset({'Books'}), confidence=0.938131313131313, lift=3.8029923981669307)]), RelationRecord(items=frozenset({'UnderwearAndSocks', 'FridgeMagnets'}), support=0.16744532803180914, ordered_statistics=[OrderedStatistic(items_base=frozenset({'FridgeMagnets'}), items_add=frozenset({'UnderwearAndSocks'}), confidence=0.6533501405992437, lift=3.5532922905410955), OrderedStatistic(items_base=frozenset({'UnderwearAndSocks'}), items_add=frozenset({'FridgeMagnets'}), confidence=0.9106636031896203, lift=3.5532922905410955)])]
[RelationRecord(items=frozenset({'FlagsAndStandards', 'Books'}), support=0.13848161033797216, ordered_statistics=[OrderedStatistic(items_base=

In [87]:
data = []
# print("Support Confidence Rule")
for a in results:
    support = a.support
    items= a.items
    rules = a.ordered_statistics
    # print(rules)
    for b in rules:
        lhs = b.items_base
        rhs = b.items_add
        conf = b.confidence
        lift = b.lift
        data.append([support, conf, lhs, rhs, lift])
        print("%6.4f %6.4f %s ---> %s %.2f" %(support, conf, lhs, rhs, lift))


0.1385 0.5614 frozenset({'Books'}) ---> frozenset({'FlagsAndStandards'}) 3.80
0.1385 0.9381 frozenset({'FlagsAndStandards'}) ---> frozenset({'Books'}) 3.80
0.1674 0.6534 frozenset({'FridgeMagnets'}) ---> frozenset({'UnderwearAndSocks'}) 3.55
0.1674 0.9107 frozenset({'UnderwearAndSocks'}) ---> frozenset({'FridgeMagnets'}) 3.55


In [88]:
columns = ["Support", "Confidence", "LHS", "RHS", "Lift"]
data1 = pd.DataFrame(data, columns = columns)
data1

,Support,Confidence,LHS,RHS,Lift
0,0.138482,0.561376,(Books),(FlagsAndStandards),3.802992
1,0.138482,0.938131,(FlagsAndStandards),(Books),3.802992
2,0.167445,0.653350,(FridgeMagnets),(UnderwearAndSocks),3.553292
3,0.167445,0.910664,(UnderwearAndSocks),(FridgeMagnets),3.553292


In [89]:
data1.nlargest(n=10, columns="Confidence")


,Support,Confidence,LHS,RHS,Lift
1,0.138482,0.938131,(FlagsAndStandards),(Books),3.802992
3,0.167445,0.910664,(UnderwearAndSocks),(FridgeMagnets),3.553292
2,0.167445,0.653350,(FridgeMagnets),(UnderwearAndSocks),3.553292
0,0.138482,0.561376,(Books),(FlagsAndStandards),3.802992


In [90]:
df = df.drop('ID', axis=1)

In [ ]:
from mlxtend.frequent_patterns import apriori, association_rules

In [92]:
frequent_itemssets = apriori(df, min_support=0.1, use_colnames=True)
frequent_itemssets['length'] = frequent_itemssets['itemsets'].apply(lambda x: len(x))

d:\Users\jk026110\assosiaatio\env\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [98]:
rules = association_rules(frequent_itemssets,
                          metric="lift",
                          min_threshold=0.1,
                          num_itemsets=100000)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(FlagsAndStandards),(Books),0.11880,0.19853,0.11145,0.938131,4.725388,1.0,0.087865,12.954372,0.894663,0.541335,0.922806,0.749754
1,(Books),(FlagsAndStandards),0.19853,0.11880,0.11145,0.561376,4.725388,1.0,0.087865,2.009011,0.983664,0.541335,0.502243,0.749754
2,(UnderwearAndSocks),(FridgeMagnets),0.14798,0.20626,0.13476,0.910664,4.415125,1.0,0.104238,8.884845,0.907849,0.613997,0.887449,0.782007
3,(FridgeMagnets),(UnderwearAndSocks),0.20626,0.14798,0.13476,0.653350,4.415125,1.0,0.104238,2.457869,0.974508,0.613997,0.593144,0.782007


In [99]:
rules.nlargest(n=10, columns="confidence")

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(FlagsAndStandards),(Books),0.11880,0.19853,0.11145,0.938131,4.725388,1.0,0.087865,12.954372,0.894663,0.541335,0.922806,0.749754
2,(UnderwearAndSocks),(FridgeMagnets),0.14798,0.20626,0.13476,0.910664,4.415125,1.0,0.104238,8.884845,0.907849,0.613997,0.887449,0.782007
3,(FridgeMagnets),(UnderwearAndSocks),0.20626,0.14798,0.13476,0.653350,4.415125,1.0,0.104238,2.457869,0.974508,0.613997,0.593144,0.782007
1,(Books),(FlagsAndStandards),0.19853,0.11880,0.11145,0.561376,4.725388,1.0,0.087865,2.009011,0.983664,0.541335,0.502243,0.749754
